In [16]:
import numpy as np
import os
import argparse
import pandas as pd
from itertools import combinations_with_replacement 
from itertools import permutations 
from tqdm.notebook import tqdm
import csv
from cvxopt import cvxopt
import cvxpy as cp


### --------------------------------- Kernel method---------------------------

In [20]:
# quadratic problem solver
def cvxopt_solve_qp(P, q, G=None, h=None, A=None, b=None):
    P = .5 * (P + P.T)
    args = [cvxopt.matrix(P), cvxopt.matrix(q)]
    if G is not None:
        args.extend([cvxopt.matrix(G), cvxopt.matrix(h)])
        if A is not None:
            args.extend([cvxopt.matrix(A), cvxopt.matrix(b)])
    sol = cvxopt.solvers.qp(*args)
    if 'optimal' not in sol['status']:
        return None
    return (np.array(sol['x']).reshape((P.shape[1],))) #,sol['primal objective']


def ridge_reg(K,Y,lamda):
    
    """
      inputs:
           K: Gram matrix: n*n pd matrix
           Y:the data labels: n*1 vector
           lamda: the regularization scalar
        -----------------------------------
        outputs:
           Alpha: vector that contains the representer theorem parameters: n*1 vector
    """
    num_samples=len(K)
    Alpha=np.linalg.solve((K + lamda*num_samples*np.eye(num_samples)),Y)
    return(Alpha)

def my_SVM(K,Y,lamda):
    """
      inputs:
           K: Gram matrix: n*n pd matrix
           Y:the data labels: n*1 vector
           lamda: the regularization scalar
        -----------------------------------
        outputs:
           Alpha: vector that contains the representer theorem parameters: n*1 vector
    """
    P=2*K
    q= ((-2*Y).reshape(len(Y),1)).astype('float')
    G=np.concatenate((np.diag(Y), -np.diag(Y)), axis=0).astype('float')
    h=np.concatenate(((1/(2*lamda*len(Y)))*np.ones(len(Y)), np.zeros(len(Y))), axis=0).reshape(2*len(Y),1).astype('float')
    Alpha=cvxopt_solve_qp(P, q, G, h)
    return(Alpha)
    

### -------------------------------------------- Data extraction---------------------------------

In [7]:
Xtr0 = pd.read_csv('Xtr0.csv', sep=',', header=None).to_numpy()[1:,1]
Xtr0_mat=pd.read_csv('Xtr0_mat100.csv', sep=',', header=None).to_numpy()[1:,:]
Ytr0_str=pd.read_csv('Ytr0.csv', sep=',', header=None).to_numpy()[1:]
Ytr0=np.array([int(Ytr0_str[i][1]) for i in range(len(Ytr0_str))])

Xtr1 = pd.read_csv('Xtr1.csv', sep=',', header=None).to_numpy()[1:,1]
Xtr1_mat=pd.read_csv('Xtr1_mat100.csv', sep=',', header=None).to_numpy()[1:,:]
Ytr1_str=pd.read_csv('Ytr1.csv', sep=',', header=None).to_numpy()[1:]
Ytr1=np.array([int(Ytr1_str[i][1]) for i in range(len(Ytr1_str))])

Xtr2 = pd.read_csv('Xtr2.csv', sep=',', header=None).to_numpy()[1:,1]
Xtr2_mat=pd.read_csv('Xtr2_mat100.csv', sep=',', header=None).to_numpy()[1:,:]
Ytr2_str=pd.read_csv('Ytr2.csv', sep=',', header=None).to_numpy()[1:]
Ytr2=np.array([int(Ytr2_str[i][1]) for i in range(len(Ytr2_str))])

Xte0 = pd.read_csv('Xte0.csv', sep=',', header=None).to_numpy()[1:,1]
Xte0_mat=pd.read_csv('Xte0_mat100.csv', sep=',', header=None).to_numpy()[1:,:]

Xte1 = pd.read_csv('Xte1.csv', sep=',', header=None).to_numpy()[1:,1]
Xte1_mat=pd.read_csv('Xte1_mat100.csv', sep=',', header=None).to_numpy()[1:,:]

Xte2 = pd.read_csv('Xte2.csv', sep=',', header=None).to_numpy()[1:,1]
Xte2_mat=pd.read_csv('Xte2_mat100.csv', sep=',', header=None).to_numpy()[1:,:]



### --------------------------------- Spectrum Kernel --------------------

In [8]:
def spectrum(k,x):
    '''
    this function compute the spectrum of length k for a sequence x
    ----------------------------------------------------------------
    inputs:
        k: legth of the spectrum
        x: sequence in question
    ----------------------------------------------------------------
    outputs:
        L_spec: liste containing the spectrum
    '''
    L_spec=[]
    for i in range(len(x)-k+1):
        L_spec.append(x[i:i+k])
    return(L_spec)


    
list_char=['A','C','G','T'] # list of possible characters that may appear in the DNA 
def General_spec(k,list_char):
    '''
    this function gives the complete possible spectrum using letters in list_char
    -----------------------------------------------------------------------------
    inputs:
        k:legth of the spectrum
        list_char: list of possible letters
    -----------------------------------------------------------------------------
    outputs:
        list_spec_comp: list of the complete general spectrum
    '''
    list_spec_comp=[]
    comb = combinations_with_replacement(list_char, k)
    for i in list(comb):
         for j in list(permutations(list(i))):
                    word=''.join(list(j))
                    list_spec_comp.append(word)
    list_spec_comp=np.unique(list_spec_comp, axis=0)
    return(list_spec_comp)
  
    
def New_feature_vector(list_spec_comp,spec_x):
    '''
    inputs:
        list_spec_comp: list of the complete general spectrum
        spec_x:list containing the spectrum of x
    ---------------------------------------------------------
    outputs:
        feature_vector: vector which contains the new representation of x
    '''
    feature_vector=[]
    for u in list_spec_comp:
        feature_vector.append(spec_x.count(u))
        
    return(np.array(feature_vector))

def New_feature_matrix(k,X,list_spec_comp):
    New_feature_matrix=np.zeros((len(X),len(list_spec_comp)))
    for i in tqdm(range(len(X))):
        New_feature_matrix[i,:]=New_feature_vector(list_spec_comp,spectrum(k,X[i]))
    return(New_feature_matrix)
    
    

def spectrum_kernel(k,x,y,list_spec_comp):
    '''
    inputs:
        k: length of the spectrum
        x,y: two vector in the X space
    --------------------------------------------------
    outputs:
        K(x,y):<phi(x),phi(y)>
    '''
    
    spec_x=spectrum(k,x)
    spec_y=spectrum(k,y)
    K_x_y=np.sum(New_feature_vector(list_spec_comp,spec_x)*New_feature_vector(list_spec_comp,spec_y))
    
    return(K_x_y)


def Gram_spectrum_kernel(k,X):
    list_spec_comp=General_spec(k,list_char)
    feature_matrix= New_feature_matrix(k,X,list_spec_comp)
    K_mat=np.dot(feature_matrix,feature_matrix.T)
    return(K_mat + 0.0000001*np.eye(len(X)),feature_matrix) #+ 0.0000001*np.eye(len(X)) I add that to get rid of the very tiny negative eigenvalues
    
def predict(X_test,feature_matrix_tr,Alpha,k):
    New_feature_mat_X_test=New_feature_matrix(k,X_test,list_spec_comp)
    Gram_mat=np.dot(New_feature_mat_X_test,feature_matrix_tr.T)
    Y=np.dot(Gram_mat,Alpha)
    Y_=np.zeros_like(Y)
    Y_[Y>0]=1
    return(Y_)
    

### --------------------------------------------------------k=8----------------------------------------------------------------------------

In [9]:
k=8 # kength of the spectrum
list_char=['A','C','G','T'] # list of possible characters that may appear in the DNA
list_spec_comp=General_spec(k,list_char)


K_tr0,feature_matrix_tr0=Gram_spectrum_kernel(k,Xtr0)
K_tr1,feature_matrix_tr1=Gram_spectrum_kernel(k,Xtr1)
K_tr2,feature_matrix_tr2=Gram_spectrum_kernel(k,Xtr2)

lamda=0.1
Alpha_svm_0=my_SVM(K_tr0,Ytr0*2-1,lamda)
Alpha_svm_1=my_SVM(K_tr1,Ytr1*2-1,lamda)
Alpha_svm_2=my_SVM(K_tr2,Ytr2*2-1,lamda)


Alpha_ridge_reg_0=ridge_reg(K_tr0,Ytr0*2-1,lamda)
Alpha_ridge_reg_1=ridge_reg(K_tr1,Ytr1*2-1,lamda)
Alpha_ridge_reg_2=ridge_reg(K_tr2,Ytr2*2-1,lamda)


Yte0_svm=predict(Xte0,feature_matrix_tr0,Alpha_svm_0,k)
Yte1_svm=predict(Xte1,feature_matrix_tr1,Alpha_svm_1,k)
Yte2_svm=predict(Xte2,feature_matrix_tr2,Alpha_svm_2,k)

Yte0_ridge=predict(Xte0,feature_matrix_tr0,Alpha_ridge_reg_0,k)
Yte1_ridge=predict(Xte1,feature_matrix_tr1,Alpha_ridge_reg_1,k)
Yte2_ridge=predict(Xte2,feature_matrix_tr2,Alpha_ridge_reg_2,k)




  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

     pcost       dcost       gap    pres   dres
 0: -2.1256e+01 -2.6078e+01  5e+03  7e+01  1e-15
 1: -2.1214e+01 -2.5141e+01  3e+02  5e+00  1e-15
 2: -1.8960e+01 -1.8841e+01  1e+02  1e+00  8e-16
 3: -1.1069e+01 -1.2756e+01  2e+01  2e-01  6e-16
 4: -7.5284e+00 -9.6937e+00  2e+00  3e-17  8e-16
 5: -7.8079e+00 -8.0876e+00  3e-01  2e-17  4e-16
 6: -7.8647e+00 -7.8962e+00  3e-02  2e-17  3e-16
 7: -7.8765e+00 -7.8782e+00  2e-03  2e-17  4e-16
 8: -7.8772e+00 -7.8773e+00  1e-04  2e-17  4e-16
 9: -7.8772e+00 -7.8772e+00  2e-06  2e-17  4e-16
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -2.0960e+01 -2.5742e+01  4e+03  7e+01  8e-16
 1: -2.0934e+01 -2.4971e+01  2e+02  3e+00  7e-16
 2: -1.8494e+01 -1.7448e+01  5e+01  6e-01  7e-16
 3: -9.0080e+00 -1.2279e+01  8e+00  5e-02  1e-15
 4: -8.2692e+00 -8.9409e+00  7e-01  9e-04  5e-16
 5: -8.4731e+00 -8.5403e+00  7e-02  9e-05  3e-16
 6: -8.4960e+00 -8.5003e+00  5e-03  5e-06  3e-16
 7: -8.4976e+00 -8.4977e+00  1e-04  1e-07  3e-1

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

In [121]:
Yte_svm=np.concatenate((Yte0_svm,Yte1_svm,Yte2_svm),axis=0).astype(int)
Yte_svm_list=[['id','bound']]
for i in range(len(Yte_svm)):
    Yte_svm_list.append([i,Yte_svm[i]])    
with open('Yte_svm.csv', 'w', newline='') as file:
    writer = csv.writer(file, delimiter=',')
    writer.writerows(Yte_svm_list)

Yte_ridge=np.concatenate((Yte0_ridge,Yte1_ridge,Yte2_ridge),axis=0).astype(int)
Yte_ridge_list=[['id','bound']]
for i in range(len(Yte_ridge)):
    Yte_ridge_list.append([i,Yte_ridge[i]])    
with open('Yte_ridge.csv', 'w', newline='') as file:
    writer = csv.writer(file, delimiter=',')
    writer.writerows(Yte_ridge_list)

### ------ testing on the training data---------------------------

In [12]:
Ytr0_test=np.dot(K_tr0,Alpha_svm_0)
Y=np.zeros_like(Ytr0_test)
Y[Ytr0_test>0]=1
score=len(np.where(Y==Ytr0)[0])
score/=2000
print('score svm',score)

Ytr0_test=np.dot(K_tr0,Alpha_ridge_reg_0)
Y=np.zeros_like(Ytr0_test)
Y[Ytr0_test>0]=1
score=len(np.where(Y==Ytr0)[0])
score/=2000
print('score ridge',score)

score svm 0.967
score ridge 0.9865


In [11]:
# test
Ytr1_test=np.dot(K_tr1,Alpha_svm_1)
Y=np.zeros_like(Ytr1_test)
Y[Ytr1_test>0]=1
score=len(np.where(Y==Ytr1)[0])
score/=2000
print('score svm',score)

Ytr1_test=np.dot(K_tr1,Alpha_ridge_reg_1)
Y=np.zeros_like(Ytr1_test)
Y[Ytr1_test>0]=1
score=len(np.where(Y==Ytr1)[0])
score/=2000
print('score ridge',score)

score svm 0.985
score ridge 0.9985


In [13]:
Ytr2_test=np.dot(K_tr2,Alpha_svm_2)
Y=np.zeros_like(Ytr2_test)
Y[Ytr2_test>0]=1
score=len(np.where(Y==Ytr2)[0])
score/=2000
print('score svm',score)

Ytr2_test=np.dot(K_tr2,Alpha_ridge_reg_2)
Y=np.zeros_like(Ytr2_test)
Y[Ytr2_test>0]=1
score=len(np.where(Y==Ytr2)[0])
score/=2000
print('score ridge',score)

score svm 0.92
score ridge 0.993
